In [1]:
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
import os
langchain.__version__


'0.3.27'

In [2]:
with open("data/secrets/key.secret") as f:
    os.environ["OPENAI_API_KEY"] = f.read()

In [3]:
model = init_chat_model("gpt-4", model_provider="openai")
model

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000018A1523D940>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000018A1523E3C0>, root_client=<openai.OpenAI object at 0x0000018A14FFEF90>, root_async_client=<openai.AsyncOpenAI object at 0x0000018A1523E120>, model_name='gpt-4', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [4]:
# Ne fonctionne pas car non chainé
model.invoke([{"role": "system", "content": "Traduis ce texte du Français à l'Italien"}])
response=model.invoke([{"role": "user", "content": "Salut, je m'appelle Cyril"}])
response.content

"Bonjour Cyril, comment puis-je vous aider aujourd'hui?"

In [5]:
# Chainage
system_message={"role": "system", "content": "Traduis ce texte du Français à l'Italien"}
user_message={"role": "user", "content": "Salut, je m'appelle Cyril"}
response=model.invoke([system_message,user_message])
response.content

'Ciao, mi chiamo Cyril'

In [6]:
# Autre écriture avec une logique OO Langchain
messages = [
    SystemMessage(content="Traduis ce texte du Français à l'Italien"),
    HumanMessage(content="Salut, je m'appelle Cyril"),
]
response=model.invoke(messages)
response.content

'Ciao, mi chiamo Cyril'

In [7]:
# Template
system_template = "Traduis ce texte du Francais vers {language}"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),  ("user", "{text}")]
)
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Traduis ce texte du Francais vers {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [8]:
prompt = prompt_template.invoke({"language": "Italian", "text": "Salut, je m'appelle Cyril"})
prompt

ChatPromptValue(messages=[SystemMessage(content='Traduis ce texte du Francais vers Italian', additional_kwargs={}, response_metadata={}), HumanMessage(content="Salut, je m'appelle Cyril", additional_kwargs={}, response_metadata={})])

In [9]:
response = model.invoke(prompt)
response.content

'Ciao, mi chiamo Cyril.'

HUB RLM
Templates préfabriqués sur le cloud

rlm.rag-prompt : https://smith.langchain.com/hub/rlm/rag-prompt?_gl=1*11dcy42*_gcl_au*MTAxOTgzMzI4My4xNzYwMzQ5ODk5*_ga*MTI3MDkwMDAzMS4xNzYwMzQ5ODk5*_ga_47WX3HKKY2*czE3NjA2MzYyOTEkbzgkZzEkdDE3NjA2MzczMzkkajI0JGwwJGgw

In [10]:
prompt_template = hub.pull("rlm/rag-prompt")
prompt_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [11]:
prompt = prompt_template.invoke({"context": "Le Hub RLM est une base de données de ChatPromptTemplate déjà écrits et disponibles sur https://smith.langchain.com/hub/rlm/, répondre en 5 mots", "question": "Qu'est que le HUB RLM ?"})
response = model.invoke(prompt)
response.content

'Le HUB RLM est une base de données de ChatPromptTemplate disponible en ligne.'